
<img src="../../img/ods_stickers.jpg">

## <center>Открытый курс по машинному обучению. Сессия № 2</center> ##
# <center>Индивидуальный проект по анализу данных </center>#
### <center>Автор материала: Кирилл Власов (@vlasoff)</center> ###



##  Часть 0. Импорт библиотек

In [ ]:
import numpy as np
import pandas as pd
import scipy.stats as st
import sklearn
from sklearn import linear_model,metrics,model_selection, preprocessing
from tqdm import tqdm_notebook 
from matplotlib import pyplot as plt
import geopy as geo
import geopy.distance as dist
import arcgis
from arcgis.gis import GIS
import folium
from folium import plugins
import plotly.plotly as py
import plotly.graph_objs as go
import seaborn as sns

#Отключим предупредения
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

##  Часть 1. Описание набора данных и признаков

### Цели и задачи ###
Спрогнозировать кол-во посещений в музеях РФ в 2016 году по историческим данным. Сделать выводы о влиянии различных факторов на посещаемость. 
> Прим.: Известны данные с 2008 по 2016 год, поэтому будем прогнозировать именно 2016, однако, "отрежем" данные за этот год, позже - после некоторых преобразований

### Источник данных ###

Для исследования использованны статистические данные по музеям с официального сайта <a href ="http://opendata.mkrf.ru/opendata/7705851331-stat_museum/">Министерства Культуры России</a>
Данные за последние 9 лет можно скачать в удобном для работы csv формате, однако сайт не позволяет скачивать данные сразу за несколько лет, поэтому на входе получены 9 файлов. Для удобства работы названия признаков заменены на короткие. Итогововый датасет можно скачать <a href ="https://drive.google.com/open?id=1BMKpbFUymeW4f16vc-HoChNxHewjJcXm">тут</a>.

### Признаки ###
#### Целевой ####
- __visitor:__ Число посещений – всего, измеряется в тыс.чел  

#### Объясняющие ####
- __Name:__ Название музея  
- __kopuk:__ КОПУК (код классификатора по учреждениям культуры)
- __adres:__ Адрес  
- __pred_general:__ Число предметов основного фонда на конец года, единиц  
- __pred_second:__ Число предметов научно-вспомогательного фонда на конец года, единиц  
- __pred_elcat:__ Число музейных предметов, внесенных в электронный каталог музея, единиц  
- __ex:__ Число выставок – всего, единиц  
- __workers:__ Численность работников - всего, человек  
- __build:__ Число строений - всего, единиц  


In [ ]:
#Соберем DataFrame из нескольких csv файлов. 
for i in range(9):
    data = pd.read_csv("../../data/museum_research/data-" + str(i+1) + "-structure-1.csv",)
    data['year'] = 2008 + i #добавим еще признак, отражающий ГОД, на конец которого представлены данные
    if i == 0:
        df = data
    else:  
        df = pd.concat([df,data],axis=0, ignore_index=True)

In [ ]:
print('DataFrame содержит ' + str(df.shape[0]) +' объектов и ' + str(df.shape[1]) + ' признаков. Взглянем на них:')
df.head()

В глаза бросается адрес Музея, как кладязь информации. 
Попробуем сразу получить больше информации из признака "Адрес", сгенерируем дополнительно признаки из географических координат:  
- __longitude__: долгота
- __latitude__: широта  
Попутно из геоданных заберем пару категориальных признаков:
- __region__: Название Субъекта
- __okrug__: Название округа
  
Для геокодирования призанов, воспользуемся классом geocode из библиотеки __ArcGis__

In [ ]:
gis = GIS()

uni_adress = pd.unique(df['adres']) #выберем уникальные Адреса
lat = {} #сформируем словарь для широты
long = {} #сформируем словарь для долготы
region = {}
okrug = {}

#Пробежимся по нашему списку уникальных адресов
for i in tqdm_notebook(range(uni_adress.shape[0])): 
    geo = arcgis.geocoding.geocode(uni_adress[i], max_locations=1)[0] #получаем ГЕО данные 
    lat[str(uni_adress[i])] = geo['location']['y'] #формируем словари с координатами (x - долгота, y - широта)
    long[str(uni_adress[i])] = geo['location']['x']
    region[str(uni_adress[i])] = geo['attributes']['Region']
    okrug[str(uni_adress[i])] = geo['attributes']['Territory']
    
df['latitude'] = df['adres']
df['longitude'] = df['adres']
df['region'] = df['adres']
df['okrug'] = df['adres']

df['latitude'] = df['latitude'].apply(lambda x: lat[x])
df['longitude'] = df['longitude'].apply(lambda x: long[x])
df['region'] = df['region'].apply(lambda x: region[x])
df['okrug'] = df['okrug'].apply(lambda x: okrug[x])

df.to_csv('../../data/museum_research/data-all-coord.csv',index_label='index') #сохраним данные

> _Вычисления выше, занимают от 30 минут до часа, чтобы не собирать геоданные заного, данные сохранены в файл 'data-all-coord.csv'_

## Пришло время, разделить данные на обучающую и контрольную выборки ##

Напомню, что прогнозировать планируем посещения в 2016-ом году.
>_Но сначала загрузим данные заного, если выше, что-то пошло не так :)_

In [ ]:
df = pd.read_csv('../../data/museum_research/data-all-coord.csv',
                 index_col='index') 

df_pred = df[df.year == 2016]
df = df[df.year != 2016]
print('Объектов для исследования: ', df.shape[0],', для прогноза:', df_pred.shape[0])
print('Итоговый датасет содержит: ',df.shape[1], ' признаков, в том числе 1 целевой')

Проводить исследование данных будем, принимая тот факт, что мы вообще не знаем данных за 2016 год, а лишь будем смотреть на нее как на отложенную выборку. 

##  Часть 2. Первичный анализ данных ##
Разобьем признаки на типы и поговорим про каждый из них
- Целевой 
    - visitor
- Категориальные признаки: 
    - kopuk
    - region 
    - okrug
    - year
- Вещественные признаки: 
    - pred_general 
    - pred_second
    - pred_elcat
    - ex
    - workers
    - build
    - latitude
    - longitude
- Текстовые признаки: 
    - Name
    - adres

In [ ]:
#Категориальные признаки: 
cat = ['kopuk','region','okrug','year']
#Вещественные признаки:
num = ['pred_general', 'pred_second', 'pred_elcat', 'ex', 'workers',
       'build', 'latitude', 'longitude']
#Текстовые признаки:
txt = ['Name','adres']

### Изучение целевого признака ###
Целевая переменая, вещественное число и наша задача будет сведена к задаче восстановления регрессии, поэтому нам важно, чтобы целевая переменная имела нормальное распределение. 

Построим графики плотности рапределения целевой пременной, а так же сравним распредление с нормальным. 

In [ ]:
sns.set(rc={"figure.figsize": (20, 5)})
fig = plt.figure()
ax1 = fig.add_subplot(121)
p_x = sns.distplot(df['visitor'], fit=st.norm, kde=True,ax=ax1, bins=50)
ax1.legend()
ax1.set_title('Плотность распределения целевой переменной')
ax2 = fig.add_subplot(122)
prob = st.probplot(df['visitor'], dist=st.norm, plot=ax2)
ax2.set_title('Probabilyty plot')
ax2.legend()
plt.show()

Из графиков становится видно, что распределение не симметрично, скошено и совсем не походит на нормальное распределение, Дополнительно проверим это статистическими тестами с помощью класса stats из библиотеки scipy, но для начала вспомним теорию.
> _Распределение оценивается как предположительно близкое к нормальному, если установлено, что от 50 до 80 % всех значений располагаются в пределах одного стандартного отклонения от среднего арифметического, и коэффициент эксцесса по абсолютной величине не превышает значения равного двум._

> _Распределение считается достоверно нормальным если абсолютная величина показателей асимметрии и эксцесса меньше их ошибок репрезентативности в 3 и более раз._

> _Дополнительно применим тест Шапиро-Уилка_

In [ ]:
def norm_testing(vector):
    testA = np.sqrt(6/len(vector))
    testE = 2 * np.sqrt(6/len(vector))
    A = np.abs(np.round(st.skew(vector),decimals=2))
    E = np.abs(np.round(st.kurtosis(vector) ,decimals=2))
    print('Асимметрия: ', A, '. Скошенность больше ошибки репрезентативности в : ',
        np.abs(np.round(st.kurtosis(vector)/testA, decimals=2)) , ' раз.')
    print('Эксцесс: ', E, '. Куртозис больше ошибки репрезентативности в : ',
          np.abs(np.round(st.kurtosis(vector)/testE, decimals=2)) , ' раз.')
    print('Гипотеза о нормальности распределения не отвергается с вероятностью: ', st.shapiro(vector)[1]*100, '%')
    return 

norm_testing(df['visitor'].values)

### Вывод 1: ###
__Распределение не нормально__, повторюсь, что перед нами стоит задача восстановения регресии, то нам очень важно приблизить распределение целевой переменной к нормальному. (См. часть "Предобработка данных")

### Изучение категориальных признаков ###

In [ ]:
df[cat].head(3)

In [ ]:
for i in range(len(cat)):
    print('Признак', cat[i], ' содержит ', len(np.unique(df[cat[i]].fillna(""))), ' уникальных значений')
    
#fillna применён, потому что в признаке okrug есть какое-то пустое значение,
# вызваное следствием ошибки в геокоддировании (См. часть "Предобработка данных")

- признак kopuk, исходя из своей природы, является уникальным ID для каждого музея и имеет место  OneHotEncoding
- признаки region и okrug заданы текстово и перед применением OneHotEncoding необходимо произвести label-кодирование
- Признак year в рамках проекта будем рассматривать как вещественный, тем более, что в на отложенной выборке знаечение 2016 появляется впервые.

In [ ]:
#Категориальные признаки: 
cat = ['kopuk','region','okrug']
#Вещественные признаки:
num = ['pred_general', 'pred_second', 'pred_elcat', 'ex', 'workers',
       'build', 'latitude', 'longitude','year']

### Вывод 2: ###
__Проведем OneHotEncoding по всем категориальным признакам, кроме year __

### Изучение вещественных признаков ###

In [ ]:
df[num].describe()

### Вывод 3: ###
Вещественные признаки имеют разную размерность, поэтому необходимо произвести их стандартизацию 
### Изучение Текстовых признаков ###

In [ ]:
df[txt].head(3)

- Название музея является таким же уникальным ID для каждого музея как kopuk, и будет с ним корреллировать,
- Из признака adres, мы получили достаточно информации на этапе сбора датасета,
В рамках текущего проекта предлагается данне признак не использовать.

Однако, в дальнейшем их можно использовать в качестве источника для порождения дополнительных признаков (см. часть "Выводы")

##  Часть 3. Первичный визуальный анализ данных ##

В рамках проекта решается задача прогнозирования посещений в 2016-ом году, посмотрим, на тенденцию изменения посещений по Годам.

In [ ]:
pvt = df.pivot_table(columns=['year'], values='visitor', aggfunc='sum')
plt.figure()
sns.barplot(pvt.columns.values, pvt.values[0], color='Orange', palette=None,)
plt.xlabel('Год')
plt.ylabel('Кол-во посещений, тыс. чел.')
plt.show()

In [ ]:
print('Взглянем, на матрицу корреляции признаков:')
sns.set(palette='GnBu_r', rc={"figure.figsize": (20, 5)})
sns.heatmap(df.corr(), vmin=0, vmax=1,annot=True, cmap='Greens',)
plt.show()

### Выводы: ###
- Признак pred_elcat имеет сильную линейную связь (r=0.82) с pred_general, что вполне логично, исходя из природы данных. Возникает риск мультиколлениарности. 
- координаты музея, слабо коррелируют с целевым признаком. 
- целевой признка имеет наибольшую линейную связь (r=0.73) с workers, однако здесь нужно учесть, что скорее всего именно кол-во посещений влияло на кол-во сотрудников. 



In [ ]:
sns.pairplot(df)

Стоит сделать предположение, что координаты, в некоторых строках определены не верно и данные являются выбросами

Попробуем построить HeatMap посещений на карте РФ по координатам с помощью модуля Folium

In [ ]:
tunguzka = [60.7308, 101.9744]
folium.Map.choropleth
h_map = folium.Map(location=tunguzka, zoom_start=3,tiles='OpenStreetMap', width='100%', height = 700)

data = [[x[0], x[1],x[2]] for x in np.array(df[df['year'] ==2015][['latitude','longitude','visitor']])]

plugins.HeatMap(data, radius = 12,blur = 9).add_to(h_map)
h_map.render()
#h_map.save('map.html')
h_map

Судя по карте, можно предположить, что кол-во посещений сильно зависит от расстояния до центральных городов, как например Москва и Петербург и может послужить дополнительным признаком (см. часть "Создание новых признаков и описание этого процесса")

##  Часть 4. Закономерности, "инсайты", особенности данных##

### Выбросы, пропуски, ошибки Гео-данных###
Посмотрим, что из себя представляют выбросы на координатах:

In [ ]:
print('Всего объектов:', df[df.latitude <= 25]['region'].shape[0],
      ' в регионах:', np.unique(np.unique(df[df.latitude <= 25]['region'])))

print('Всего объектов:', df[df.longitude <= 0]['region'].shape[0],
      ' в регионах:', np.unique(np.unique(df[df.longitude <= 0]['region'])))

Для Чукотского автономного округа такие координаты являются вполне нормальными, однако, Сингапура в РФ точно нет, судя по Wiki :)  

Посмотрим, что это за музей втесался в наши официальные данные МинКульта:

In [ ]:
np.unique(df[df.region == 'Singapore']['Name'])

Ошибка в гео данных "выскочила" по одному музею, и можно без особого труда восстановить данные (см. часть "Предобработка данных")
Правильные значения (ист. Wiki):
- okrug: 'Центральный федеральный округ'
- latitude: 55.7744
- longitude: 37.6395
- region: 'Москва'   




### Проверка на выбросы по целевому признаку ###
Проверим, что за аномальные скачки были у целевого признака и являются ли они выбросами

In [ ]:
un_name = np.unique(df[df.visitor > 3000]['Name']) 
for i in range(len(un_name)):
    print(un_name[i], ' за год посещают в среднем: ', 
          np.round(np.mean(df[df.Name == un_name[i]]['visitor']), decimals=2)
          , 'тыс., а максимум достигал: ',
            np.round(np.max(df[df.Name == un_name[i]]['visitor']), decimals=2), 'тыс.')

Так как в выборку попали самые крупные музеи РФ, то смело можно сделать вывод, что данные скачки обусловленны природой данных, а не аномальными выбросами.

##  Часть 5. Выбор метрики##

В задачах регрессии обычно используется среднеквадратичная ошибка или средняя абсолютная ошибка, которые подходят для сравнения двух моделей или для контроля качества во время обучения, но не позволяет сделать выводов о том, насколько хорошо данная модель решает задачу.
Учитывая широкий разброс целевой переменной, и специфику задачи логично в качестве метрики выбрать коэффициент детерминации (r2_score) и посмотреть насколько хорошо модель объясняет данные или же наш прогноз сопоставим по качеству с константным предсказанием.

##  Часть 6. Выбор модели ##

Предлагается использовать линейную модель для восстановления регрессии.  

__Предпосылки: __
- Большое кол-во признаков, с разряженными матрицам
- Хочется интерпретировать коэфициенты для выявления влияния факторов

__Ограничения: __
- Имеем риск получения плохого результата, так как зависимость целевой переменной от признаков нелинейная (Видно из матрицы корреляции)

В качестве линейной модели будет использован __SGDRegressor__:
- Быстро обучается
- Гибко настраивается
- Позволяет использовать регуляризацию

##  Часть 7. Предобработка данных ##


### Исправление ошибок Геокодинга ###


In [ ]:
df[df.region == 'Singapore']['okrug'] = 'Центральный федеральный округ'
df[df.region == 'Singapore']['latitude'] = 55.7744
df[df.region == 'Singapore']['longitude'] = 37.6395
df[df.region == 'Singapore']['region'] = 'Москва'

df_pred[df_pred.region == 'Singapore']['okrug'] = 'Центральный федеральный округ'
df_pred[df_pred.region == 'Singapore']['latitude'] = 55.7744
df_pred[df_pred.region == 'Singapore']['longitude'] = 37.6395
df_pred[df_pred.region == 'Singapore']['region'] = 'Москва'

# !!!!!!!!!!!! Не присваиваются значения! Исправь! 
df.dropna(inplace=True)
df_pred.dropna(inplace=True)

### Обработка целевого признака для приведения к нормальному распределению ###
Попробуем для начала просто взять логарифм из целевой переменной

In [ ]:
target_modified = np.log((df['visitor'].values)+1)
norm_testing(target_modified) #Используем тест, который использовали ранее

#Построим графики
fig = plt.figure()
ax1 = fig.add_subplot(121)
prob = sns.distplot(target_modified, fit=st.norm, kde=True,ax=ax1)
ax1.set_title('Гистограмма распределения целевой переменной')
ax2 = fig.add_subplot(122)
prob = st.probplot(target_modified, dist=st.norm, plot=ax2)
ax2.set_title('Probabilyty plot')
ax2.legend()
plt.show()

Логарифмирование дало эффект, однако, для достижения большего результата применим Метод Бокса-Кокса, так же из класса stats из библиотеки scipy

In [ ]:
step_boxcox = 0.592
target_modified = (df['visitor']+step_boxcox)
par = st.boxcox(target_modified)
target_modified = par[0]
par_lamb = par[1] #Пригодится, для изменения тестовой выборки
norm_testing(target_modified) #Используем тест, который использовали ранее
df['target'] = target_modified
#Построим графики
fig = plt.figure()
ax1 = fig.add_subplot(121)
prob = sns.distplot(target_modified, fit=st.norm, kde=True,ax=ax1)
ax1.set_title('Гистограмма распределения целевой переменной')
ax2 = fig.add_subplot(122)
prob = st.probplot(target_modified, dist=st.norm, plot=ax2)
ax2.set_title('Probabilyty plot')
ax2.legend()
plt.show()

Картина стала в целом лучше, но Говорить о нормальности распредления нам не позволяет тест Шапиро-Уилка.
> _Прим.: в документации scipy к данному тесту, есть ссылка, что при более чем 5000 объектов p-value может отображаться не корректно._

Применим Трансформацию и к отложенной выборке:

In [ ]:
df_pred['target'] = df_pred['visitor']+step_boxcox
df_pred['target'] = st.boxcox(df_pred['target'].values, lmbda=par_lamb)

In [ ]:
#Удалим старую целевую переменную
df.drop('visitor', inplace=True, axis=1)
df_pred.drop('visitor', inplace=True, axis=1)

### Проведем OneHotEncoding по всем категориальным признакам ###

Так как уникальные значения категориатных признаков в 2016-ом году встречаются не все, то чтобы ничего не потерять, соединим наши выборки, а потом снова разделим.

In [ ]:
df_u = pd.concat([df, df_pred])
df_OHE = pd.concat([df_u.drop('kopuk', 1), pd.get_dummies(df_u['kopuk'],prefix='kopuk') ], axis=1).reindex()
df_OHE = pd.concat([df_OHE.drop('region', 1), pd.get_dummies(df_OHE['region'],prefix='reg') ], axis=1).reindex()
df_OHE = pd.concat([df_OHE.drop('okrug', 1), pd.get_dummies(df_OHE['okrug'],prefix='okr') ], axis=1).reindex()


### Удалим Текстовые фичи ###        

Обоснование удаления приведено в части "Первичный анализ данных"

In [ ]:
df_txt = df_OHE.drop(['Name','adres'],axis=1)

### Шкалирование вещественных признаков ###

Шкалирование вещественных признаков и повторное разбиение на тестовую и отложеную выборки будет произведено в части "Кросс-валидация, подбор параметров"


##  Часть 8. Создание новых признаков и описание этого процесса ##

Ранее выдвигалось предположение, о том, что расстояния до Москвы и Санкт-Петербурга могут влиять на кол-во посещений. За основу расчета растояний возьмем Координаты Московского кремля и Дворцовой площади

In [ ]:
kremlin = [55.750730, 37.617322] # Координаты Московского кремля
dp_piter = [59.9384, 30.3227] #Координаты Дворцовой площади
dist_to_kremlin = []
dist_to_dp = []

for index, row in tqdm_notebook(df_txt.iterrows()):
    dist_to_kremlin = np.append(dist_to_kremlin, dist.great_circle(( row['latitude'], row['longitude']),(kremlin[0],kremlin[1])).km)
    dist_to_dp = np.append(dist_to_dp, dist.great_circle(( row['latitude'], row['longitude']),(dp_piter[0],dp_piter[1])).km)
    
df_txt['dist_to_kremlin']=dist_to_kremlin
df_txt['dist_to_dp']=dist_to_dp



Рассмотрим облако рассеивания новых переменных и целевой

In [ ]:
g = sns.PairGrid(df_txt[['target', 'dist_to_kremlin','dist_to_dp']], size=5)
g = g.map(plt.scatter)

Из графика видно, что связь не линейна, чтобы в этом убедиться взглянем на матрицу корреляций:

In [ ]:
df_txt[['target', 'dist_to_kremlin','dist_to_dp']].corr()

__Итак:__ Новые признаки не имеют линейной связи, но отрицательные коэфициенты корреляции говорят о наличии обратной зависимости: Чем больше расстояние, тем меньше целевая переменная, что соответсвует нашему изначальному предположению.

## Часть 0. Финальная предобработка данных ##

In [ ]:
# Повторно разделим выборку на Обучение и Тест
df_final_pred = df_txt[df_txt.year == 2016]
df_final_train = df_txt[df_txt.year != 2016]
print('Объектов для обучения: ', df_final_train.shape[0],', для прогноза:', df_final_pred.shape[0])
print('Итоговый датасет содержит: ',df_final_train.shape[1], ' признаков, в том числе 1 целевой')

In [ ]:
X_train = df_final_train.drop('target', axis=1).values
y_train = df_final_train['target'].values

X_test = df_final_pred.drop('target', axis=1).values
y_test = df_final_pred['target'].values

#Проведем стандартизацию признаков, о котором говорили в части "Предобработка данных"
scaling = sklearn.preprocessing.StandardScaler()
X_train = scaling.fit_transform(X_train)
X_test = scaling.transform(X_test)

## Часть 9. Кросс-валидация и настройка гиперпараметров модели ##

Части "Выбор модели" было описано, что в качестве модели мы будем использовать стохастический градиентный спуск. 
Принимая во внимание, что данные имеют большие выбросы, я буду использовать функцию потерь Губера, её использование позволит уменьшить вклад выбросов и большой квадратичной ошибки, а неточности будут проигнорированы. 


Крос-валидация будет проведена на трех фолдах с перемешиванием.  
Гиперпарметрами для функции будет: 
- Alpha - определяющий порог регуляризации
- Epsilon - порог, при котором становится менее важным получить предсказание точно правильно. Тоесть когда функция потерь изменит свою работу с вида L<sub>2</sub> на L<sub>1</sub>

Обучим класс GridSearchCV у четом сказанного выше.

In [ ]:
kf = model_selection.KFold(n_splits=3, shuffle=True, random_state=18)

SGD = linear_model.SGDRegressor(loss='huber', max_iter=100, penalty='l2',
                                random_state=18,learning_rate='optimal')

params = {'alpha': np.linspace(0.01,0.04, num=5),
          'epsilon': np.linspace(0.1,2, num=5)}

grid = model_selection.GridSearchCV(SGD, param_grid=params, scoring='r2', cv=kf, verbose=3)
grid.fit(X_train,y_train)
grid.best_score_

In [ ]:
grid.best_estimator_

На кросвалидации R<sup>2</sup> - составил 0.86, при параметрах:  
- alpha = 0.04  
- epsilon = 1.525

Учитывая, что коэффициент детерминации выше 80%, можно говорить о хорошем качестве модели

## Часть 10. Построение кривых валидации и обучения ##

In [ ]:
alphas = np.linspace(0.01,0.04, num=5)
val_train, val_test = model_selection.validation_curve(SGD, X_train, y_train, param_name='alpha',
                                                       param_range=alphas, cv=kf,
                                       scoring='r2')

def plot_with_err(x, data, **kwargs):
    mu, std = data.mean(1), data.std(1)
    lines = plt.plot(x, mu, '-', **kwargs)
    plt.fill_between(x, mu - std, mu + std, edgecolor='none',
                     facecolor=lines[0].get_color(), alpha=0.2)


In [ ]:
train_sizes = np.linspace(0.05, 1, 10)
N_train, tr_train, tr_test = model_selection.learning_curve(SGD,
                                                  X_train, y_train, train_sizes=train_sizes, cv=kf,
                                                  scoring='r2')

In [ ]:
fig = plt.figure(figsize=(20, 5))
ax1 = fig.add_subplot(121)
ax1.set_title('Кривая Валидации')
plot_with_err(alphas, val_train, label='training scores')
plot_with_err(alphas, val_test, label='validation scores')
plt.xlabel(r'$\alpha$'); plt.ylabel('r^2')
plt.legend();


ax2 = fig.add_subplot(122)
ax2.set_title('Кривая Обучения')
plot_with_err(N_train, tr_train, label='training scores')
plot_with_err(N_train, tr_test, label='validation scores')
plt.xlabel('Training Set Size'); plt.ylabel('r^2')
plt.legend()

Валидационная кривая:
- Так кривые валидации близко друг к другу и их значение близко к 1, то можно говорить об отсутствии переобучения или недообучения. 

Кривая обучения:
- Так как кривые сошлись друг к другу, добавление новых данных ничего не изменит.

## Часть 11. Прогноз для тестовой выборки ##

Спрогнозируем колличество посещений в музея в 2016 году по результатам модели и расчитаем кофициент детерминации для Тестовой выборки:

In [ ]:
np.round(metrics.r2_score(y_test, grid.predict(X_test)), decimals=10)

R<sub>2</sub> Для тестовой выборки соотносится с показателем достигнутым на кросс-валидации. 
Можно сделать вывод о том, что модель имеет хорошее качество.  
Посмотрим, какие факторы максимально влияют на целевую переменую:

In [ ]:
df_coef = pd.DataFrame({ 'name': df_final_train.drop('target', axis=1).columns, 'value': grid.best_estimator_.coef_ })

In [ ]:
n_top_features = 15
coef_plot = pd.concat([df_coef.sort_values(by='value', ascending=False).head(n_top_features),
                        df_coef.sort_values(by='value', ascending=False).tail(n_top_features)], axis=0)

interesting_coefficients = coef_plot['value']
feature_names = coef_plot['name']

plt.figure(figsize=(15, 5))
colors = ["red" if c < 0 else "blue" for c in interesting_coefficients]
plt.bar(np.arange(1, 2 * n_top_features+1), interesting_coefficients, color=colors)
plt.xticks(np.arange(1, 2 * n_top_features+1), feature_names, rotation=90, ha="center")
plt.show()

##  Часть 12. Выводы ##

Создана модель, которая имеет хорошую обобщающую способность. 
Ценность модели заключается в том, что ее можно использовать для прогнозирования кол-ва посетителей в музеях. На основании полученных данных можно оптимизировать затраты. 

Кроме того, можно выделять факторы, которые максимально влияют на посещаемость музея

В части путей улучшения, можно использовать следующие направления:
- Порождение дополнительных фич из названия музея, например его категорию (Археологический, Исторический, Краеведческий) или оценку (отзывы поситителей).
- Порождатьть дополнительные фичи на основании геоданных из Адреса.
- Для повышения интерпритируемости, возможно Смещение фич на +1 Год для прогноза, так как датасет содержить данные на конец года, но формально они соответствуют данным на начало года